In [3]:
from torch.autograd import Function
from torch.autograd import Variable as V
import torch as t
import numpy as np
import math as m
import matplotlib.pyplot as plt
from IPython import display
from sklearn.preprocessing import StandardScaler

In [4]:
#fogd
class conventional_linear_regression_fractional(Function):
    def forward(ctx,w,x,b):
        output = w*x + b
        ctx.save_for_backward(w,x,b,output)
        return output
    
    def backward(ctx,grad_outputs):
        w,x,b,output = ctx.saved_tensors
        if a==1.0:
            grad_w = grad_outputs*x  #1-orde
            grad_b = grad_outputs*1   #1-order
            grad_x = grad_outputs*w  #1-order
            return grad_w,None,grad_b
             

        y_pred_f = t.abs(y_pred-y)
        grad_outputs = 0.005*0.5*m.gamma(3)*y_pred_f**(2-a)/m.gamma(3-a)
        
        #w
        w = t.abs(w)
        grad_w = grad_outputs*(x*w**(1-a)/m.gamma(2-a)+b*w**(-a)/m.gamma(1-a))  #fractional-order

        #b
        b = t.abs(b)
        grad_b = grad_outputs*(x*w*b**(-a)/m.gamma(1-a)+b**(1-a)/m.gamma(2-a))  #fractional-order

        return grad_w,None,grad_b

In [5]:
#ifogd
class linear_regression_fractional(Function):
    def forward(ctx,w,x,b):
        output = w*x + b
        ctx.save_for_backward(w,x,b,output)
        return output
    
    def backward(ctx,grad_outputs):
        w,x,b,output = ctx.saved_tensors
        if a==1.0:
            grad_w = grad_outputs*x  #1-orde
            grad_b = grad_outputs*1   #1-order
            grad_x = grad_outputs*w  #1-order
            return grad_w,None,grad_b
             
        y1 = t.where(y_pred-y <= 0, t.tensor([0.]), t.tensor([1.]))
        y_pred_f = t.abs(y_pred-y)
        grad_outputs1 = 0.5*m.gamma(3)*y_pred_f**(2-a)/m.gamma(3-a)*y1
        y2 = t.where(y_pred-y > 0, t.tensor([0.]), t.tensor([1.]))
        grad_outputs2 = (y_pred-y)*y2

        grad_outputs = 0.005*(grad_outputs1 + grad_outputs2)
        
        #w
        if w<=0:
            grad_w = grad_outputs*x  #1-order
        else:
            grad_w = grad_outputs*(x*w**(1-a)/m.gamma(2-a)+b*w**(-a)/m.gamma(1-a))  #fractional-order

        #b
        if b<=0:
            grad_b = grad_outputs*1   #1-order
        else:
            grad_b = grad_outputs*(x*w*b**(-a)/m.gamma(1-a)+b**(1-a)/m.gamma(2-a))  #fractional-order

        return grad_w,None,grad_b

In [6]:
t.manual_seed(1000) 

#random(x,y),y = 2*x - 3 + e
def get_fake_data(batch_size=200):
    x = t.linspace(-10,10,steps=batch_size).view(-1,1)
    y = 2*x - 3 + t.normal(0,5,size=(batch_size, 1))
    return x, y

In [24]:
a = 1.5   #alpha,0.1,0.3,0.5,0.7,0.9,1.0,1.1,1.3,1.5       ##

w = V(t.tensor([-1.]), requires_grad=True)  #initial w
b = V(t.tensor([-4.]), requires_grad=True)  #initial b



#loss_fogd_15=[]   #fogd，0.1,0.3,0.5,0.7,0.9,1.0,1.1,1.3,1.5    ##
loss_ifogd_15=[]   #ifogd，0.1,0.3,0.5,0.7,0.9,1.0,1.1,1.3,1.5   ##

#ww=[]
#bb=[]


lr =1e-3 # learning rate

for ii in range(100):
    x, y = get_fake_data()
    x, y = V(x), V(y)

    #scaler = StandardScaler()
    #x = t.tensor(scaler.fit_transform(x))
    #y = t.tensor(scaler.fit_transform(y))
    
    #y_pred = conventional_linear_regression_fractional.apply(w,x,b)   #2,-3   #
    y_pred = linear_regression_fractional.apply(w,x,b)   #2,-3   #
    loss = 0.5 * (y_pred - y)** 2
    loss = loss.mean() 
    
#    if loss.detach()<L_best:
#        L_best = loss.detach()
#        w_best = w.data
#        b_best = b.data
    
    #loss_fogd_15.append(loss.detach())    ##
    loss_ifogd_15.append(loss.detach())   ##


    

    loss.backward()  #fractional order grad
    
    w.data.sub_(lr * w.grad.data)
    b.data.sub_(lr * b.grad.data)
    

    
    w.grad.data.zero_()
    b.grad.data.zero_()
    
print(w.data,b.data)

tensor([1.6120]) tensor([-3.7369])


In [33]:
fig = plt.figure(figsize=(10,9),dpi=1200)

custom_params = {
    'font.size': 3,
    'lines.linewidth': 1,
    'axes.labelsize': 10,
    'axes.titlesize': 10,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 8,
    'grid.linestyle': '--',
    'grid.alpha': 0.7,
}

plt.rcParams.update(custom_params)
fig.subplots_adjust(wspace=0, hspace=0) 


ax1 = fig.add_subplot(3, 3, 1)
ax1.plot(loss_fogd_01,label=r'${FOGD}$')
ax1.plot(loss_ifogd_01,label=r'${IFOGD}$')
ax1.set_xticklabels([])
#ax1.set_xlim(0,100)
ax1.set_ylim(0,200)
ax1.set_ylabel(r'${loss}$')
ax1.set_title(r'$\alpha  = 0.1$',bbox=dict(facecolor='y', edgecolor='blue', alpha=0.65 ),x=0.5,y=0.5)
ax1.legend()
ax1.grid()

ax2 = fig.add_subplot(3, 3, 2)
ax2.plot(loss_fogd_03,label=r'${FOGD}$')
ax2.plot(loss_ifogd_03,label=r'${IFOGD}$')
ax2.set_xticklabels([])
ax2.set_yticklabels([])
#ax2.set_xlim(0,100)
ax2.set_ylim(0,200)
ax2.set_title(r'$\alpha  = 0.3$',bbox=dict(facecolor='y', edgecolor='blue', alpha=0.65 ),x=0.7,y=0.5)
ax2.legend()
ax2.grid()

ax3 = fig.add_subplot(3, 3, 3)
ax3.plot(loss_fogd_05,label=r'${FOGD}$')
ax3.plot(loss_ifogd_05,label=r'${IFOGD}$')
ax3.set_xticklabels([])
ax3.set_yticklabels([])
#ax3.set_xlim(0,100)
ax3.set_ylim(0,200)
ax3.set_title(r'$\alpha  = 0.5$',bbox=dict(facecolor='y', edgecolor='blue', alpha=0.65 ),x=0.5,y=0.5)
ax3.legend()
ax3.grid()

ax4 = fig.add_subplot(3, 3, 4)
ax4.plot(loss_fogd_07,label=r'${FOGD}$')
ax4.plot(loss_ifogd_07,label=r'${IFOGD}$')
#ax4.set_xticklabels([0,20,40,60,80,])
#ax4.set_xlim(0,100)
ax4.set_yticklabels([0,25,50,75,100,125,150,175,])
ax4.set_ylim(0,200)
ax4.set_xlabel(r'${iterations}$')
ax4.set_ylabel(r'${loss}$')
ax4.set_title(r'$\alpha  = 0.7$',bbox=dict(facecolor='y', edgecolor='blue', alpha=0.65 ),x=0.5,y=0.5)
ax4.legend()
ax4.grid()

ax5 = fig.add_subplot(3, 3, 5)
ax5.plot(loss_fogd_09,label=r'${FOGD}$')
ax5.plot(loss_ifogd_09,label=r'${IFOGD}$')
ax5.set_yticklabels([])
ax5.set_ylim(0,200)
#ax5.set_xticklabels([0,20,40,60,80,])
#ax5.set_xlim(0,100)
ax5.set_xlabel(r'${iterations}$')
ax5.set_title(r'$\alpha  = 0.9$',bbox=dict(facecolor='y', edgecolor='blue', alpha=0.65 ),x=0.5,y=0.5)
ax5.legend()
ax5.grid()

ax6 = fig.add_subplot(3, 3, 6)
ax6.plot(loss_fogd_10,label=r'${FOGD}$')
ax6.plot(loss_ifogd_10,label=r'${IFOGD}$')
ax6.set_yticklabels([])
ax6.set_ylim(0,200)
ax6.set_xticklabels([0,20,40,60,80,100])
#ax6.set_xlim(0,100)
ax6.set_xlabel(r'${iterations}$')
ax6.set_title(r'$\alpha  = 1.0$',bbox=dict(facecolor='y', edgecolor='blue', alpha=0.65 ),x=0.5,y=0.5)
ax6.legend()
ax6.grid()

ax7 = fig.add_subplot(3, 3, 7)
ax7.plot(loss_fogd_11,label=r'${FOGD}$')
ax7.plot(loss_ifogd_11,label=r'${IFOGD}$')
#ax4.set_xticklabels([0,20,40,60,80,])
#ax4.set_xlim(0,100)
ax7.set_yticklabels([0,25,50,75,100,125,150,175,])
ax7.set_ylim(0,200)
ax7.set_xlabel(r'${iterations}$')
ax7.set_ylabel(r'${loss}$')
ax7.set_title(r'$\alpha  = 1.1$',bbox=dict(facecolor='y', edgecolor='blue', alpha=0.65 ),x=0.5,y=0.5)
ax7.legend()
ax7.grid()

ax8 = fig.add_subplot(3, 3, 8)
ax8.plot(loss_fogd_13,label=r'${FOGD}$')
ax8.plot(loss_ifogd_13,label=r'${IFOGD}$')
ax8.set_yticklabels([])
ax8.set_ylim(0,200)
#ax5.set_xticklabels([0,20,40,60,80,])
#ax5.set_xlim(0,100)
ax8.set_xlabel(r'${iterations}$')
ax8.set_title(r'$\alpha  = 1.3$',bbox=dict(facecolor='y', edgecolor='blue', alpha=0.65 ),x=0.5,y=0.5)
ax8.legend()
ax8.grid()

ax9 = fig.add_subplot(3, 3, 9)
ax9.plot(loss_fogd_15,label=r'${FOGD}$')
ax9.plot(loss_ifogd_15,label=r'${IFOGD}$')
ax9.set_yticklabels([])
ax9.set_ylim(0,200)
#ax6.set_xticklabels([0,20,40,60,80,100])
#ax6.set_xlim(0,100)
ax9.set_xlabel(r'${iterations}$')
ax9.set_title(r'$\alpha  = 1.5$',bbox=dict(facecolor='y', edgecolor='blue', alpha=0.65 ),x=0.5,y=0.5)
ax9.legend()
ax9.grid()

#plt.savefig('picture/Fig3.6.eps',bbox_inches='tight',format='eps')
#plt.savefig('picture/Fig3.6.svg',bbox_inches='tight',format='svg')
#plt.savefig('picture/Fig3.6.pdf',bbox_inches='tight',format='pdf')
plt.show()

C:\Users\1508-T3630\AppData\Local\Temp\ipykernel_54460\3474647188.py:57: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax4.set_yticklabels([0,25,50,75,100,125,150,175,])
C:\Users\1508-T3630\AppData\Local\Temp\ipykernel_54460\3474647188.py:82: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax6.set_xticklabels([0,20,40,60,80,100])
C:\Users\1508-T3630\AppData\Local\Temp\ipykernel_54460\3474647188.py:94: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax7.set_yticklabels([0,25,50,75,100,125,150,175,])
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
